In [1]:
#Two datasets found - A big and small dataset provided by Movielens 
complete_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest.zip'
small_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
data_path = '/dbfs/FileStore/data'


In [2]:
import urllib
small_f = urllib.urlretrieve (small_dataset_url, '/dbfs/FileStore/small.zip')
complete_f = urllib.urlretrieve (complete_dataset_url, '/dbfs/FileStore/complete.zip')


In [3]:
import zipfile

with zipfile.ZipFile('/dbfs/FileStore/small.zip', "r") as z:
    z.extractall(data_path)

with zipfile.ZipFile('/dbfs/FileStore/complete.zip', "r") as z:
    z.extractall(data_path)

In [4]:
import os
os.listdir(data_path+'/ml-latest-small')


In [5]:
os.listdir(data_path+'/ml-latest')

In [6]:
import pandas as pd
pandas_df = pd.read_csv("/dbfs/FileStore/data/ml-latest-small/ratings.csv", header=0)
small_ratings = sqlContext.createDataFrame(pandas_df).drop('timestamp').rdd.map(tuple)
pandas_df = pd.read_csv("/dbfs/FileStore/data/ml-latest-small/movies.csv",header=0)
small_movies = sqlContext.createDataFrame(pandas_df).drop('genres').rdd.map(tuple)

#I remove timestamp from the ratings file and genres from the movies file while converting csv to RDD.

#While there was no 'real' point to first converting csv into pandas DF to then spark's RDDs; it read much easier for me.



In [7]:
small_ratings.take(3)

In [8]:
small_movies.take(2)

In [9]:
new_user_ID = 0
new_user_ratings = [
     (0,1,5), #Toy story
     (0,2,4), # Jumanji
     (0,3,3), # Grumpier old men
     (0,4,2), # Waiting to exhale
     (0,5,2), # Father of bride pt. 2
     (0,6,1), # Heat
     (0,7,5), # Sabrina 
     (0,8,4), #Tom and huck 
     (0,9,3) , # sudden death
     (0,10,3) # golden eye
    ]
small_ratings = small_ratings.union(sc.parallelize(new_user_ratings))

In [10]:
small_training,small_validation,small_test = small_ratings.randomSplit([6.0,2.0,2.0],seed = 26)
small_validation_predict = small_validation.map(lambda x: (x[0],x[1]))
small_test_predict = small_test.map(lambda x: (x[0],x[1]))
user_predictions = small_movies.map(lambda x: (x[0],x[1]))

In [11]:
small_test_predict.take(3)

In [12]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
rank = 8
numIterations = 10
model = ALS.train(small_training, rank, numIterations)

In [13]:
predictions = model.predictAll(small_validation_predict).map(lambda z: ((z[0],z[1]),z[2]))
rates_and_predictions = small_ratings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
MSE = rates_and_predictions.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error = " + str(MSE))



In [14]:
#model.save(sc, "target/tmp/myCollaborativeFilter")
#sameModel = MatrixFactorizationModel.load(sc, "target/tmp/myCollaborativeFilter")

In [15]:

test_predictions = model.predictAll(small_test_predict).map(lambda z: ((z[0],z[1]),z[2]))
test_rates_and_predictions = small_test.map(lambda r: ((r[0], r[1]), r[2])).join(test_predictions)
MSE = test_rates_and_predictions.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error = " + str(MSE))


In [16]:
#https://www.codementor.io/jadianes/building-a-recommender-with-apache-spark-python-example-app-part1-du1083qbw
'''def get_counts_and_averages(ID_and_ratings_tuple):
    nratings = len(ID_and_ratings_tuple[1])
    return ID_and_ratings_tuple[0], (nratings, float(sum(x for x in ID_and_ratings_tuple[1]))/nratings)''' #WIP

In [17]:
'''complete_data = (small_ratings.map(lambda x: (x[1], x[2])).groupByKey())
complete_data_with_added_data = complete_data.map(get_counts_and_averages)
times_rated_data = complete_data_with_added_data.map(lambda x: (x[0], x[1][0]))''' #WIP

In [18]:
times_rated_data.take(3)
# This shows frequency of movies rated

In [19]:
'''def recommend_me_a_movie_or_three(model,my_movie_profile,rating_data=small_ratings,movie_data = small_movies,times_rated_data = times_rated_data,new_user_ID=new_user_ID):
  rating_data = rating_data.union(my_movie_profile)
  movies_not_to_include = rating_data.map(lambda x: x[1], my_movie_profile)
  movies_to_recommend = (rating_data.filter(lambda x: x[0] not in movies_not_to_include).map(lambda x: (new_user_ID, x[0])))
  user_predictions = model.predictAll(movies_to_recommend)
  
  
  user_predictions_ratings = user_predictions.map(lambda x: (x.product, x.rating))
  return  user_predictions_ratings''' # WIP

In [20]:
small_ratings.take(2)

In [21]:

my_rated_movies = set([x[1] for x in new_user_ratings])
movies_to_recommend = sc.parallelize([m for m in user_predictions.collect() if m[0] not in my_rated_movies])
predictions = model.predictAll(movies_to_recommend.map(lambda x: (0,x[0])))
recommendations = sorted(predictions.collect(), key=lambda x: x[2], reverse=True)
movie_names = dict(movies_to_recommend.collect())


In [22]:

predictions.take(3)

In [23]:
recommendations[:3]

In [24]:
for x in recommendations[:3]:
  print str(movie_names[x[1]]) + ' Is recommended to me'
  

In [25]:
#Big thanks to :
#https://www.codementor.io/jadianes/building-a-recommender-with-apache-spark-python-example-app-part1-du1083qbw
#https://github.com/databricks/spark-training/blob/master/machine-learning/python/solution/MovieLensALS.py